In [2]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [3]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [5]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders
from orders.order_logs_snapshot
where yyyymmdd = '20220919'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
and customer_location_hex_8 in ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a05fffff', '8860a24a0dfffff', '8860a24a21fffff',
  '8860a24a25fffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a45fffff', '8860a24a4dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff',
   '8860a24b09fffff', '8860a24b0bfffff', '8860a24b19fffff', '8860a24b1dfffff', '8860a24b43fffff', '8860a24b49fffff', '8860a24b4bfffff', '8860a24b53fffff',
    '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25981fffff', '8860a25983fffff', '8860a25985fffff',
     '8860a25987fffff', '8860a2598bfffff', '8860a25991fffff', '8860a25993fffff', '8860a25999fffff', '8860a2599bfffff', '8860a2599dfffff', '8860a259a1fffff',
      '8860a259a3fffff', '8860a259a7fffff', '8860a259a9fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff', '8860a259d7fffff')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [6]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders
0,6327838249a7d319a06578ed,0.178,Hyderabad,8860a24b51fffff,17.465212,78.365852,021554,02,0215,Link,02:00,1,20220919,customerCancelled,Others,3.0,"[""62f692ef50b11f23f1c9ddfb"",""603fd5c2f2578a122..."
1,6327822549a7d319a065787b,0.390,Hyderabad,8860a24a61fffff,17.446398,78.363586,021005,02,0200,Link,02:00,1,20220919,customerCancelled,Wrong pickup location,6.0,"[""5d42d6f701176418cc60c75b"",""5c9e15c48c352421e..."
2,632784187ff9294e2753e373,0.119,Hyderabad,8860a24a65fffff,17.450076,78.363372,021824,02,0215,Link,02:00,1,20220919,customerCancelled,Drop location denied,3.0,"[""5ca46d2054bc7263ff24b0ae"",""63099b6be3fa3cc3b..."
3,6327811449a7d319a065782f,1.245,Hyderabad,8860a25997fffff,17.434700,78.374649,020532,02,0200,Link,02:00,1,20220919,dropped,Requested for extra passengers,4.0,"[""5d11cabad6cbfc6a9db22467"",""5f0470854975ff637..."
4,6327805205e89e76c1647bee,0.409,Hyderabad,8860a259b9fffff,17.440081,78.385750,020218,02,0200,Link,02:00,1,20220919,dropped,None,4.0,"[""5dedbc0a1a882e3354b98d2b"",""5dbcafdd340548492..."


In [7]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['map_riders'] = df['map_riders'].apply(lambda x: json.loads(x))
df = df[df['map_riders'].apply(lambda x: len(x)) > 0]
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,second,minute,date
0,6327838249a7d319a06578ed,0.178,Hyderabad,8860a24b51fffff,17.465212,78.365852,021554,02,0215,Link,02:00,1,20220919,customerCancelled,Others,3.0,"[62f692ef50b11f23f1c9ddfb, 603fd5c2f2578a1221d...",54,15,19
1,6327822549a7d319a065787b,0.390,Hyderabad,8860a24a61fffff,17.446398,78.363586,021005,02,0200,Link,02:00,1,20220919,customerCancelled,Wrong pickup location,6.0,"[5d42d6f701176418cc60c75b, 5c9e15c48c352421ea0...",05,10,19
2,632784187ff9294e2753e373,0.119,Hyderabad,8860a24a65fffff,17.450076,78.363372,021824,02,0215,Link,02:00,1,20220919,customerCancelled,Drop location denied,3.0,"[5ca46d2054bc7263ff24b0ae, 63099b6be3fa3cc3b75...",24,18,19
3,6327811449a7d319a065782f,1.245,Hyderabad,8860a25997fffff,17.434700,78.374649,020532,02,0200,Link,02:00,1,20220919,dropped,Requested for extra passengers,4.0,"[5d11cabad6cbfc6a9db22467, 5f0470854975ff637f5...",32,05,19
4,6327805205e89e76c1647bee,0.409,Hyderabad,8860a259b9fffff,17.440081,78.385750,020218,02,0200,Link,02:00,1,20220919,dropped,None,4.0,"[5dedbc0a1a882e3354b98d2b, 5dbcafdd34054849201...",18,02,19


In [8]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,second,minute,date,20_sec_batch,count
0,6327838249a7d319a06578ed,0.178,Hyderabad,8860a24b51fffff,17.465212,78.365852,021554,02,0215,Link,02:00,1,20220919,customerCancelled,Others,3.0,"[62f692ef50b11f23f1c9ddfb, 603fd5c2f2578a1221d...",54.0,15.0,19,40-60,1
1,6327822549a7d319a065787b,0.390,Hyderabad,8860a24a61fffff,17.446398,78.363586,021005,02,0200,Link,02:00,1,20220919,customerCancelled,Wrong pickup location,6.0,"[5d42d6f701176418cc60c75b, 5c9e15c48c352421ea0...",5.0,10.0,19,0-20,1
2,632784187ff9294e2753e373,0.119,Hyderabad,8860a24a65fffff,17.450076,78.363372,021824,02,0215,Link,02:00,1,20220919,customerCancelled,Drop location denied,3.0,"[5ca46d2054bc7263ff24b0ae, 63099b6be3fa3cc3b75...",24.0,18.0,19,20-40,1
3,6327811449a7d319a065782f,1.245,Hyderabad,8860a25997fffff,17.434700,78.374649,020532,02,0200,Link,02:00,1,20220919,dropped,Requested for extra passengers,4.0,"[5d11cabad6cbfc6a9db22467, 5f0470854975ff637f5...",32.0,5.0,19,20-40,1
4,6327805205e89e76c1647bee,0.409,Hyderabad,8860a259b9fffff,17.440081,78.385750,020218,02,0200,Link,02:00,1,20220919,dropped,None,4.0,"[5dedbc0a1a882e3354b98d2b, 5dbcafdd34054849201...",18.0,2.0,19,0-20,1


In [9]:
#Get the riders mapped in 20_sec_windows
df_captains_20_sec = df.groupby(['yyyymmdd', 'hour', 'minute', '20_sec_batch'])['map_riders'].apply(list).reset_index()
df_captains_20_sec.head()

,yyyymmdd,hour,minute,20_sec_batch,map_riders
0,20220919,00,0.0,0-20,"[[614ac928de864c694103ba54, 5d9ef048941a7d1c51..."
1,20220919,00,0.0,20-40,"[[632282ce0c0101761a087559, 616221397fefc5616f..."
2,20220919,00,0.0,40-60,"[[614ac928de864c694103ba54, 5d9ef048941a7d1c51..."
3,20220919,00,1.0,0-20,"[[628b552a8f9b4385d4623ab6, 62fe035c020cc0331c..."
4,20220919,00,1.0,20-40,"[[5e2c450e52f0da0aebfc4dd6, 5ff6fd9a31243a86d4..."


In [10]:
#Get the number of captains in a single list
map_riders = []
df_captains_20_sec = df_captains_20_sec.dropna()
for x in df_captains_20_sec['map_riders'].values:
    riders = [item for sublist in x for item in sublist]
    map_riders.append(riders)

unique_captains = []
for x in map_riders:
    unique_captains_list = []
    for y in x:
        if y in unique_captains_list:
            continue
        else:
            unique_captains_list.append(y)
    unique_captains.append(unique_captains_list)

df_captains_20_sec['total_captains'] = unique_captains
df_captains_20_sec.head()

,yyyymmdd,hour,minute,20_sec_batch,map_riders,total_captains
0,20220919,00,0.0,0-20,"[[614ac928de864c694103ba54, 5d9ef048941a7d1c51...","[614ac928de864c694103ba54, 5d9ef048941a7d1c51d..."
1,20220919,00,0.0,20-40,"[[632282ce0c0101761a087559, 616221397fefc5616f...","[632282ce0c0101761a087559, 616221397fefc5616f8..."
2,20220919,00,0.0,40-60,"[[614ac928de864c694103ba54, 5d9ef048941a7d1c51...","[614ac928de864c694103ba54, 5d9ef048941a7d1c51d..."
3,20220919,00,1.0,0-20,"[[628b552a8f9b4385d4623ab6, 62fe035c020cc0331c...","[628b552a8f9b4385d4623ab6, 62fe035c020cc0331c8..."
4,20220919,00,1.0,20-40,"[[5e2c450e52f0da0aebfc4dd6, 5ff6fd9a31243a86d4...","[5e2c450e52f0da0aebfc4dd6, 5ff6fd9a31243a86d4b..."


In [11]:
#Get the number of captains in each batch
df_captains_20_sec['number_of_captains'] = df_captains_20_sec['total_captains'].apply(lambda x: len(x))

#Get the average number of captains coming in each 20_second_window
df_20_sec_average = df_captains_20_sec.groupby(['yyyymmdd', 'hour'])['number_of_captains'].median().reset_index()
df_20_sec_average

,yyyymmdd,hour,number_of_captains
0,20220919,00,110.5
1,20220919,01,76.0
2,20220919,02,44.5
3,20220919,03,41.0
4,20220919,04,28.0
5,20220919,05,28.0
6,20220919,06,99.0
7,20220919,07,203.0
8,20220919,08,199.0
9,20220919,09,163.0


In [12]:
q_captain = """
select 
        order_id, updated_epoch, yyyymmdd, pickup_location_hex_8, captain_location_hex_8, drop_location_hex_8 as hex_id,
        city_name, order_type, captain_id, event_type, order_status, hhmmss as time, substr(hhmmss,1,2) as hour, substr(hhmmss,3,4) as min_1,
        (case when cast(substr(hhmmss,5,6) as real) >= 0 and cast(substr(hhmmss,5,6) as real) <= 20 then '0-20'
        when cast(substr(hhmmss,5,6) as real) >= 20 and cast(substr(hhmmss,5,6) as real) <= 40 then '20-40'
        when cast(substr(hhmmss,5,6) as real) >= 40 and cast(substr(hhmmss,5,6) as real) <= 60 then '40-60' end) as sec_window,
        cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count,
        array_agg(event_type) over (partition by order_id) as event_type_agg, updated_hhmmss
        
from 
        orders.order_logs_immutable
where 
        yyyymmdd = '20220919'
        and pickup_location_hex_8 IN ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
         '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a63fffff',
          '8860a24a69fffff', '8860a24a6bfffff', '8860a24b53fffff', '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff',
           '8860a25983fffff', '8860a25987fffff', '8860a25991fffff', '8860a25993fffff', '8860a2599dfffff', '8860a259a3fffff', '8860a259abfffff', '8860a259b3fffff',
            '8860a259b5fffff', '8860a259b7fffff') and order_type IN ('app')
"""

#Load data into pandas table
df_captain_copy = pd.read_sql(q_captain, connection)

In [13]:
df_captain = df_captain_copy.copy()
df_captain.head()

,order_id,updated_epoch,yyyymmdd,pickup_location_hex_8,captain_location_hex_8,hex_id,city_name,order_type,captain_id,event_type,order_status,time,hour,min_1,sec_window,map_riders_count,event_type_agg,updated_hhmmss
0,632764b77ff9294e2753d12d,1663527113234,20220919,8860a259b9fffff,8860a259b1fffff,8860a259b1fffff,Hyderabad,app,617589034c6ba12516ddfe25,rider_rated,dropped,000431,00,0431,20-40,72,"[rider_rated, order_requested, dropped, starte...",002153
1,632764b77ff9294e2753d12d,1663526071355,20220919,8860a259b9fffff,,8860a259b1fffff,Hyderabad,app,None,order_requested,requested,000431,00,0431,20-40,0,"[rider_rated, order_requested, dropped, starte...",000431
2,632764b77ff9294e2753d12d,1663527074879,20220919,8860a259b9fffff,8860a259b1fffff,8860a259b1fffff,Hyderabad,app,617589034c6ba12516ddfe25,dropped,dropped,000431,00,0431,20-40,72,"[rider_rated, order_requested, dropped, starte...",002114
3,632764b77ff9294e2753d12d,1663526478658,20220919,8860a259b9fffff,8860a259b9fffff,8860a259b1fffff,Hyderabad,app,617589034c6ba12516ddfe25,started,started,000431,00,0431,20-40,72,"[rider_rated, order_requested, dropped, starte...",001118
4,632764b77ff9294e2753d12d,1663526072263,20220919,8860a259b9fffff,,8860a259b1fffff,Hyderabad,app,,order_create,new,000431,00,0431,20-40,72,"[rider_rated, order_requested, dropped, starte...",000432


In [14]:
captains_median = df_captain.groupby(['hour', 'sec_window'])['map_riders_count'].median().reset_index()
captains_median.groupby(['hour'])['map_riders_count'].median()

hour
00     70.0
01     63.0
02     42.0
03     41.0
04     29.0
05     21.0
06     59.0
07    113.0
08     41.0
09     17.0
10     67.0
11    389.0
12      3.0
13      3.0
14      3.0
15      3.0
16      3.0
17      3.0
18      3.0
19      3.0
20      3.0
21      3.0
22      2.0
23      9.0
Name: map_riders_count, dtype: float64